In [1]:
#import libraries

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import folium #for maps
import json # library to handle JSON files

!conda install -c conda-forge geocoder --yes
import geocoder

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py36h5fab9bb_0         141 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         336 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geoco

In [10]:
#get data from wiki as html page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)

#cut out postalcode table from html page
startTable=results.text.find('<table class="wikitable sortable">\n') #where data i need starts
EndTable=results.text.find('</table>') #where data i need ends
tableData=results.text[startTable:EndTable]

#get column header names
tHeaders=tableData[tableData.find('<tr>\n<th>'):tableData.find('</th></tr>')]
tHeaders=tHeaders.replace('<tr>','').replace('</th>','').replace("\n", "")
hList=tHeaders.split('<th>')
hList.pop(0)

#get table data into list
tData=tableData[tableData.find('</th></tr>\n')+11:tableData.find('</tr></tbody>')+5]
tList=tData.replace("\n", "").split('</tr>')
i=0
#dData=pd.DataFrame
for n in tList:
    tList[i]=tList[i].replace('<tr>','')
    tList[i]=tList[i].replace('</td>','')
    tList[i]=tList[i].split('<td>')
    tList[i].pop(0)
    i=i+1
tList = list(filter(None, tList))

# create dataframe
dData=pd.DataFrame(tList)
dData.columns=hList

if dData.shape[0]==0:
    dData=pd.read_csv('borough.csv')

dData.head()    

#dData.to_csv('borough.csv', index=False) 

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
#remove not assigned postal code to Borough
dData = dData[dData.Borough != 'Not assigned']

if (dData[dData.Neighbourhood == 'Not assigned'].shape[0]==0): 
    print('no need to assign borough names to neighborough ')
dData.set_index('Postal Code',inplace=True)
dData=dData.reset_index()
#dData.tail()

no need to assign borough names to neighborough 


In [12]:
#get csv file because I couldnot get geodata from geocoder

!wget -q -O 'toronto_data.csv' 'http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
print('Data downloaded!')

Data downloaded!


In [13]:
geocsv_df = pd.read_csv('toronto_data.csv')
geocsv_df.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
geocsv_df.head(4)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917


In [14]:
i=dData.shape[0]
while i>0:
    i=i-1
    s=dData.loc[i,'Postal Code']    
    idx=geocsv_df.index[geocsv_df['PostalCode']==s].tolist()
    dData.loc[i,'Latitude']=geocsv_df.loc[idx[0],'Latitude']
    dData.loc[i,'Longitude']=geocsv_df.loc[idx[0],'Longitude']
#dData.head(12)

In [15]:
CLIENT_ID = 'DZVJPHGG0GPRF02NL5YMO341MLFPFPJTOQZXR0NJTCD3ZKHI' # your Foursquare ID
CLIENT_SECRET = '22EGVOPH4Q3VNSX25WCYDPXWIBPLTZOBDZ10SAYE2CVGFENJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#torontoData = dData[dData.Borough == 'Downtown Toronto']
torontoData = dData#[dData['Borough'].isin(['Downtown Toronto','Central Toronto','East Toronto','West Toronto'])]
#torontoData = dData[dData['Borough'].isin(['North York','York','East York'])]

torontoData.set_index('Postal Code',inplace=True)
torontoData=torontoData.reset_index()

print(torontoData.shape[0])

torontoData.head()

103


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [16]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
        LIMIT=100    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_venues=pd.DataFrame(data=getNearbyVenues(names=torontoData['Postal Code'],
                                   latitudes=dData['Latitude'],
                                   longitudes=dData['Longitude']))
print(toronto_venues.shape)

toronto_venues.head()

(2141, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [20]:
groups_df=pd.read_csv('out2_utf.csv', sep=';')

groups_df.head()

,1,2
0,Hockey Arena,entertaiment
1,Portuguese Restaurant,Restaurant
2,Coffee Shop,food
3,Intersection,Intersection
4,Pizza Place,food


In [26]:
i=0
VenueGroup=[]
#print (toronto_venues.shape[0])

while i<toronto_venues.shape[0]: 
    ValueToLook=toronto_venues.loc[i,'Venue Category']
    t=groups_df.loc[groups_df['1'] == ValueToLook ,'2'].to_list()

    try:
        k=t[0]
    except:
       k=' '
    VenueGroup.insert(i,k)
    
    i=i+1

toronto_venues['Venue Group']=VenueGroup
toronto_venues.head()    

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park,outdoor
1,M3A,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant,Restaurant
2,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop,food
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena,entertaiment
4,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop,food


In [35]:
#print(toronto_venues['Venue Group'].unique())
#lets count restaurant-group venue, food-group venue,entertaiment-group venue sport-group venue for every neighborhood 

names=toronto_venues['Venue Group'].unique()
#lets see what did we get here
print ('Groups names before')
print (names)
# not all of theese venue group are actually related to our task. lets drop some
names=['Restaurant','outdoor', 'food', 'entertaiment' , 'business',
 'store' ,'hotel'      ]

print ('Groups names after')
print (names)

Groups names before
['outdoor' 'Restaurant' 'food' 'entertaiment' 'Intersection'
 'Distribution Center' 'health and wellness' 'alco' 'market' ' ' 'store'
 'business' 'sport' 'College Auditorium' 'health' 'beauty' 'hotel'
 'College Rec Center' 'Lounge' 'Hookah Bar' 'Poutine Place' 'transport'
 'Neighborhood' 'Poke Place' 'Gas Station' 'Dog Run' 'Deli / Bodega'
 'Speakeasy' 'IT Services' 'Roof Deck' 'Grocery' 'General Travel'
 'Construction & Landscaping' 'Auto Garage' 'Gaming Cafe' 'Airport Lounge'
 'Airport Service' 'chirch' 'Garden Center' 'Escape Room']
Groups names after
['Restaurant', 'outdoor', 'food', 'entertaiment', 'business', 'store', 'hotel']


In [36]:
#preparing dataset 
neighborhood=torontoData['Postal Code']

#neighborhood=pd.DataFrame(['M3A'])

dfRes=pd.DataFrame()

tmpL=[]
for nbh in neighborhood:
    tmpList=[]
    tmpList.append(nbh)
    for name in names:
        tmp=toronto_venues[((toronto_venues['Venue Group']==name) & (toronto_venues['Neighborhood']==nbh))]
        cnt=tmp.agg(['count'])['Venue'] 
        tmpList.append(cnt[0])
    tmpL.append(tmpList)
    #print(tmpL)  
dfRes=pd.DataFrame(tmpL)
cc=[]
cc=names#.tolist()
cc.insert(0,"Neighborhood")
dfRes.columns=cc
#dfRes.head()
print (dfRes.describe())

       Restaurant     outdoor        food  entertaiment    business  \
count  103.000000  103.000000  103.000000    103.000000  103.000000   
mean     4.699029    0.766990    5.621359      1.203883    0.359223   
std      7.350917    1.021378    8.126592      2.853822    0.639460   
min      0.000000    0.000000    0.000000      0.000000    0.000000   
25%      0.000000    0.000000    0.000000      0.000000    0.000000   
50%      1.000000    0.000000    2.000000      0.000000    0.000000   
75%      4.500000    1.000000    7.000000      1.000000    1.000000   
max     29.000000    6.000000   32.000000     20.000000    2.000000   

            store       hotel  
count  103.000000  103.000000  
mean     2.951456    0.388350  
std      4.572678    1.104652  
min      0.000000    0.000000  
25%      0.000000    0.000000  
50%      1.000000    0.000000  
75%      5.000000    0.000000  
max     30.000000    6.000000  


In [38]:
#now we have everything to analyze, so lets do it
# set number of clusters
kclusters = 5
clustering = dfRes.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
#kclusters=kmeans.labels_
kmeans.labels_[0:10] 

array([3, 3, 0, 1, 0, 3, 3, 3, 3, 4], dtype=int32)

In [39]:
# add clustering labels
dfmerged=dfRes
dfmerged['Cluster Labels']=kmeans.labels_ # to analyze clusters
dfmerged['Longitude']=torontoData['Longitude']
dfmerged['Latitude']=torontoData['Latitude']

dfmerged.head()

,Neighborhood,Restaurant,outdoor,food,entertaiment,business,store,hotel,Cluster Labels,Longitude,Latitude
0,M3A,1,1,1,0,0,0,0,3,-79.329656,43.753259
1,M4A,1,0,1,1,0,0,0,3,-79.315572,43.725882
2,M5A,4,3,15,6,1,6,0,0,-79.360636,43.654260
3,M6A,1,0,1,1,0,8,0,1,-79.464763,43.718518
4,M7A,5,1,12,3,1,1,0,0,-79.389494,43.662301


In [57]:
dfmerged.loc[dfmerged['Cluster Labels'] == 3]

,Neighborhood,Restaurant,outdoor,food,entertaiment,business,store,hotel,Cluster Labels,Longitude,Latitude
0,M3A,1,1,1,0,0,0,0,3,-79.329656,43.753259
1,M4A,1,0,1,1,0,0,0,3,-79.315572,43.725882
5,M9A,0,0,0,0,0,0,0,3,-79.532242,43.667856
6,M1B,1,0,0,0,0,0,0,3,-79.194353,43.806686
7,M3B,2,0,0,0,0,0,0,3,-79.352188,43.745906
8,M4B,0,0,4,0,1,0,0,3,-79.309937,43.706397
10,M6B,3,1,2,0,0,0,0,3,-79.445073,43.709577
11,M9B,1,0,1,0,0,0,0,3,-79.554724,43.650943
12,M1C,0,0,0,0,0,0,0,3,-79.160497,43.784535
14,M4C,0,1,0,0,0,1,0,3,-79.318389,43.695344


In [41]:
# create map with marked regions
address = 'Toronto'

longitude = -79.194353
latitude  = 43.806600

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))

rainbow = ['blue','red','yellow','green','orange']


map_regions = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood,cluster in zip(dfmerged['Latitude'], dfmerged['Longitude'],  dfmerged['Neighborhood'],  dfmerged['Cluster Labels']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7,
        parse_html=False).add_to(map_regions)  
    
map_regions

In [51]:
print (dfmerged.loc[dfmerged['Cluster Labels'] == 0].describe())
print (dfmerged.loc[dfmerged['Cluster Labels'] == 1].describe())
print (dfmerged.loc[dfmerged['Cluster Labels'] == 2].describe())
print (dfmerged.loc[dfmerged['Cluster Labels'] == 3].describe())
print (dfmerged.loc[dfmerged['Cluster Labels'] == 4].describe())
#print (dfmerged.loc[dfmerged['Cluster Labels'] == 0].mean())


       Restaurant    outdoor       food  entertaiment   business      store  \
count   13.000000  13.000000  13.000000     13.000000  13.000000  13.000000   
mean    10.307692   1.153846  11.923077      1.769231   0.692308   5.692308   
std      4.327906   0.898717   3.377907      1.877669   0.751068   3.092588   
min      4.000000   0.000000   7.000000      0.000000   0.000000   1.000000   
25%      8.000000   1.000000  10.000000      1.000000   0.000000   5.000000   
50%     10.000000   1.000000  12.000000      1.000000   1.000000   5.000000   
75%     13.000000   2.000000  13.000000      2.000000   1.000000   6.000000   
max     17.000000   3.000000  19.000000      6.000000   2.000000  13.000000   

           hotel  Cluster Labels  Longitude   Latitude  
count  13.000000            13.0  13.000000  13.000000  
mean    0.230769             0.0 -79.389920  43.679836  
std     0.438529             0.0   0.037989   0.038441  
min     0.000000             0.0 -79.484450  43.644771  
25%